# Bagging和随机森林学习目标：
- 知道Bagging算法和基本原理
- 掌握sklearn中随机森林API使用
---

# 1、随机森林是什么？

![随机森林](../img/随机森林.png)


# 2、什么是bootstrap抽样？

![boostrap抽样公示](../img/boostrap抽样公示.png)

概念：Bootstrap是一种统计学上用估计一个统计量分布的方法，它不依赖于总体分布假设，是一种非参数的方法。
步骤：
    - 重抽样：从原始的样本中随机的，有放回的抽取样本，生成一个新的样本集，其大小与原始样本相同
    - 计算统计量：对新新生成的样本集计算所需的统计量
    - 重复多次：重复上述的步骤多次，每次生成新的样本集并计算统计量
    - 构建分布：将所有计算出的统计量值进行汇总，形成统计量的抽样分布
    
适合场景：Bootstrap抽样特别适合于延本量较小或者总体分布未知的情况

# 3、什么是Bagging算法？
对于一个给定含有N个训练样本的数据集D={x1, x2, ....., xn}
- 采用bootstrap有放回的抽样，组成含有n'个样本（n'<=n）新的训练集;
- 重复1中进行T遍得到T个训练集Si;
- 在每个训练集上采用某种分类算法独立的训练处T个基分类器Ci;
- 对于每个测试延本新，利用上述T个分类器得到T个预测值Ci(x);
- 对于每个x，采用多数投票的方式得到最终的预测结果C*（x）
Bagging基本流程：通过上述自助采样，采出T个含m个训练样本的采样集，然后基于每个采样集训练出一个基学习器，在将这些基学习器进行组合。

Bagging算法首先采用M轮自助采样法，获得M个包含N个训练样本的采样集。然后，基于这些采样集训练出一个基学习器。最后将这M个基学习器进行组合。组合策略为：
- 分类任务采用简单投票法：即每个基学习器一票
- 回归问题使用简单平均法：即每个基学习器的预测值取平均值

# 4、Bagging性能如何？
- Bagging是一个很高效的集成学习算法
- Bagging与下面讲的AdaBoost只适用于二分类不同，它能不经修改地用于多分类、回归任务。
- 自助bootstrap采样过程还给Bagging带来了另一个优点：由于每个基学习器只使用了初始训练集中约63.2%的样本，剩下的约36.8%样本可用作验证集来泛化性能进行“包外样本评估（即：不同于训练数据的样本）”。
- 从偏差-方差分解角度看，Bagging主要关注降低方差，因此他在不剪枝决策树、神经网络等易受样本扰动的学习器上效果更为明显。

# 4、sklearn中随机森林API有哪些？
```python
'''
    函数说明：
    参数：
        n_estimators:决策树数量 default = 10
        Criterion：entropy、gini default = gini
        max_depth：决策树最大深度 default = None
        max_features：auto、aqrt、log2、None 决策树构建时使用的最大特征数量  defalut = auto
        bootstrap：是否采用有放回抽样，如果为False将会使用全部训练样本 default = Ture
        min_samples_split:节点分裂所需最小样本数 defalue = 2
            如果节点样本数少于min_samples_split，则不会再进行划分.
            如果样本量不大，不需要设置这个值.
            如果样本量数量级非常大，则推荐增大这个值.
        min_samples_leaf:叶子节点最小样本数量 default = 1
            如果某叶子几点最小样本数，则会个兄弟节点一起被剪纸
            较小的叶子节点样本数量使模型跟容易捕捉训练数据中的噪声
        min_impurity_split: 节点划分最小不纯度 default = 1e-7
            如果某个节点的不纯度（基尼系数，均方差）小于这个阈值，则该节点不在生成子节点，病变为叶子节点

'''
sklearn.ensemble.RandomForestClassifier()

```

# 总结
- Bagging通过自助法进行采样并用于弱学习器，最后采用平权投票方式决定未知样本的最后预测
- 随机森林通过自助发、特征采样方法训练弱学习器，最后采用平权投票方式决定未知样本最后的预测
- 随机森林和Bagging关系：随机森林是基于Bagging思想实现的一种集成学习算法，它采用决策树模型作为每一个基学习器

In [36]:
# 导入pandas包
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 1、加载数据
titanic = pd.read_csv('../file/titanic.csv')
titanic.info()

# 2、选择对应列数据
x_field_arr = ['pclass', 'age', 'sex']
X = titanic[x_field_arr]
y = titanic['survived']
print(f'特征值：X:{X[:2]}, 目标值：{y[:2]}')

# 3、数据填补
X['age'].fillna(X['age'].mean(), inplace = True)
# 上述报错方法修复
# X['age'] = X['age'].replace(X['age'].mean(), np.nan, inplace=True)
# X.loc[:, ['age']] = X.loc[:, ['age']].fillna(X['age'].mean())

# 对每一个类别创建对应的列
X = pd.get_dummies(X)

# 4、训练测试集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 22)
print(f'训练集 特征值：{X_train[:2]}, 目标值：{y_train[:2]}')
print(f'测试集 特征值：{X_test[:2]}, 目标值：{y_test[:2]}')

# 5、使用单一决策树进行模型训练和预测分析
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc_predict = dtc.predict(X_test)
dtc.score(X_test, y_test)

# 6、随机森林模型进行训练和预测分析
rfc = RandomForestClassifier(max_depth = 6, random_state = 9)
rfc.fit(X_train, y_train)
rfc_predict = rfc.predict(X_test)
rfc.score(X_test, y_test)

# 7、性能评估
print(f'dtc_report:{classification_report(dtc_predict, y_test)}')
print(f'rfc_report:{classification_report(rfc_predict, y_test)}')




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB
特征值：X:   pclass   age     sex
0       3  22.0    male
1       1  38.0  female, 目标值：0    0
1    1
Name: surviv

/tmp/ipykernel_3274/4221315743.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(X['age'].mean(), inplace = True)


dtc_report:              precision    recall  f1-score   support

           0       0.86      0.79      0.82       146
           1       0.66      0.77      0.71        77

    accuracy                           0.78       223
   macro avg       0.76      0.78      0.77       223
weighted avg       0.79      0.78      0.78       223

rfc_report:              precision    recall  f1-score   support

           0       0.83      0.79      0.81       140
           1       0.67      0.72      0.69        83

    accuracy                           0.76       223
   macro avg       0.75      0.75      0.75       223
weighted avg       0.77      0.76      0.76       223



In [42]:
# 超参数选择
#导入网格搜索包
from sklearn.model_selection import GridSearchCV
# 1、实例化随机森林
rf = RandomForestClassifier()

# 2、定义超参数列表
param_dict = {
    'n_estimators': [80, 100, 200],
    'max_depth': [2, 4, 6, 8, 10, 12],
    'random_state': [9]
}

# 3、进行网格搜索
gc = GridSearchCV(rf, param_grid = param_dict, cv = 2)
gc.fit(X_train, y_train)
print(f'随机森林预测准确率为：{gc.score(X_test, y_test)}')


随机森林预测准确率为：0.7623318385650224
